In [16]:
import os,sys
import numpy as np
import cv2
from PIL import Image
from multiprocessing import Pool
import argparse
from IPython import embed #to debug
import skvideo.io
import scipy.misc

import matplotlib.pyplot as plt

# befire using optflow package to calculate optical flow --> install below openCV package

# pip install opencv-contrib-python 
# pip install sk-video
# pip install scikit-video

def save_flows(flows,image,save_dir,num,bound):
    '''
    To save the optical flow images and raw images
    :param flows: contains flow_x and flow_y
    :param image: raw image
    :param save_dir: save_dir name (always equal to the video id)
    :param num: the save id, which belongs one of the extracted frames
    :param bound: set the bi-bound to flow images
    :return: return 0
    '''

    #rescale to 0~255 with the bound setting
    # flow_x=ToImg(flows[...,0],bound)

    flow_x = flows[...,0]
    flow_x[flow_x > bound] = bound
    flow_x[flow_x < -bound] = -bound
    flow_x -=- bound
    flow_x *= (255/float(2*bound))

    # flow_y=ToImg(flows[...,1],bound)

    flow_y = flows[...,1]
    flow_y[flow_y > bound] = bound
    flow_y[flow_y < -bound] = -bound
    flow_y -=- bound
    flow_y *= (255/float(2*bound))
    
    #save the image
    save_img=os.path.join(save_dir,'img_{:05d}.jpg'.format(num))
    plt.imsave(save_img,image)

    #save the flows
    save_x=os.path.join(save_dir,'flow_x_{:05d}.jpg'.format(num))
    save_y=os.path.join(save_dir,'flow_y_{:05d}.jpg'.format(num))
    flow_x_img=Image.fromarray(flow_x)
    flow_y_img=Image.fromarray(flow_y)
   
    plt.imsave(save_x,flow_x_img)
    plt.imsave(save_y,flow_y_img)
    return 0

def dense_flow():
    '''
    To extract dense_flow images
    :param augs:the detailed augments:
        video_name: the video name which is like: 'v_xxxxxxx',if different ,please have a modify.
        save_dir: the destination path's final direction name.
        step: num of frames between each two extracted frames
        bound: bi-bound parameter
    :return: no returns
    '''

    video_path = "../vid_data/train/catch/Ball_hochwerfen_-_Rolle_-_Ball_fangen_(Timo_3)_catch_f_cm_np1_ri_med_1.avi"
    save_dir = "//PATH TO STORE OPTICAL FLOW//"
    step = 1
    bound = 15

    try:
        videocapture=skvideo.io.vread(video_path)
    except:
        print('Video Read error! ')
        return 0

    # if extract nothing, exit!
    if videocapture.sum()==0:
        print('Could not initialize capturing Video')
        exit()
    len_frame=len(videocapture)
    frame_num=0
    image,prev_image,gray,prev_gray=None,None,None,None
    num0=0
    while True:
        #frame=videocapture.read()
        if num0>=len_frame:
            break
        frame=videocapture[num0]
        num0+=1
        if frame_num==0:
            image=np.zeros_like(frame)
            gray=np.zeros_like(frame)
            prev_gray=np.zeros_like(frame)
            prev_image=frame
            prev_gray=cv2.cvtColor(prev_image,cv2.COLOR_RGB2GRAY)
            frame_num+=1
            # to pass the out of stepped frames
            step_t=step
            while step_t>1:
                #frame=videocapture.read()
                num0+=1
                step_t-=1
            continue

        image=frame
        gray=cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
        frame_0=prev_gray
        frame_1=gray

        # default choose the tvl1 algorithm
        TVL1 = cv2.optflow.DualTVL1OpticalFlow.create()
        flowDTVL1 = TVL1.calc(frame_0, frame_1, None)

        save_flows(flowDTVL1,image,save_dir,frame_num,bound) #this is to save flows and img.
        prev_gray=gray
        prev_image=image
        frame_num+=1
        # to pass the out of stepped frames
        step_t=step
        while step_t>1:
            #frame=videocapture.read()
            num0+=1
            step_t-=1


if __name__ =='__main__':

    dense_flow()
